In [1]:
!pip install plotly
!pip install lightgbm
!pip install --upgrade pip
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
from IPython.display import Image, display
from pandas.tools.plotting import scatter_matrix 
from sklearn.model_selection import train_test_split
%matplotlib inline
from sklearn import preprocessing
import lightgbm as lgb 
import datetime

Requirement already up-to-date: pip in /usr/local/lib/python3.6/site-packages (18.1)


/usr/local/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [8]:
train_df = pd.read_csv("/Users/sandy/Documents/ENPM808/Project/Github/google-revenue-prediction/train.csv")
test_df = pd.read_csv("/Users/sandy/Documents/ENPM808/Project/Github/google-revenue-prediction/test.csv")

print(max(train_df['date']))
#train_df["date"] = pd.to_datetime(train_df["date"], format="%Y%m%d")

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


20180430


In [3]:
def add_time_features(df):
    df['date'] = pd.to_datetime(df['date'], format='%Y%m%d', errors='ignore')
    df['year'] = df['date'].apply(lambda x: x.year)
    df['month'] = df['date'].apply(lambda x: x.month)
    df['day'] = df['date'].apply(lambda x: x.day)
    df['weekday'] = df['date'].apply(lambda x: x.weekday())
    
    return df

In [11]:
train_df = add_time_features(train_df)
test_df = add_time_features(test_df)

In [27]:
train_df['totals.sessionQualityDim'].isnull().sum()
train_df.shape


(1708337, 42)

In [35]:
# Impute 0 for missing target values
train_df["totals.transactionRevenue"].fillna(0, inplace=True)
train_y = train_df["totals.transactionRevenue"].values
train_id = train_df["fullVisitorId"].values
test_df["totals.transactionRevenue"].fillna(0, inplace=True)
test_y = test_df["totals.transactionRevenue"].values
test_id = test_df["fullVisitorId"].values
test_id = test_df["fullVisitorId"].values


# label encode the categorical variables and convert the numerical variables to float 
# scikit.rf needs numerical data. One hot encoding is not good on rf.
cat_cols = ["channelGrouping", "device.browser", 
            "device.deviceCategory", "device.operatingSystem", 
            "geoNetwork.city", "geoNetwork.continent", 
            "geoNetwork.country", "geoNetwork.metro",
            "geoNetwork.networkDomain", "geoNetwork.region", 
            "geoNetwork.subContinent", "trafficSource.adContent", 
            "trafficSource.adwordsClickInfo.adNetworkType", 
            "trafficSource.adwordsClickInfo.gclId", 
            "trafficSource.adwordsClickInfo.page", 
            "trafficSource.adwordsClickInfo.slot", "trafficSource.campaign",
            "trafficSource.keyword", "trafficSource.medium", 
            "trafficSource.referralPath", "trafficSource.source",
            'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.isTrueDirect']

for col in cat_cols:
    print(col)
    lbl = preprocessing.LabelEncoder()
    #lbl.fit(list(train_df[col].values.astype('str')))
    
    lbl.fit(list(train_df[col].values.astype('str')) + list(test_df[col].values.astype('str')))
    train_df[col] = lbl.transform(list(train_df[col].values.astype('str')))
    test_df[col] = lbl.transform(list(test_df[col].values.astype('str')))


num_cols = ["totals.pageviews", "visitNumber", "visitStartTime", 'totals.bounces',  'totals.newVisits']    
for col in num_cols:
    train_df[col] = train_df[col].astype(float)
    test_df[col] = test_df[col].astype(float)

# Split the train dataset into development and valid based on time 
dev_df = train_df[train_df['date']<=datetime.date(2018, 1, 1)]
dev_df = dev_df[dev_df['date']>=datetime.date(2017, 1, 1)]
val_df = train_df[train_df['date']>datetime.date(2018, 1, 1)]
dev_y = np.log1p(dev_df["totals.transactionRevenue"].values)
val_y = np.log1p(val_df["totals.transactionRevenue"].values)


cols_to_exclude = ['totals.transactionRevenue','totals.totalTransactionRevenue','totals.transactions','date','fullVisitorId']
dev_X = dev_df.copy()
val_X = val_df.copy()
test_X = test_df.copy()
dev_X.drop(cols_to_exclude, axis=1, inplace=True)
val_X.drop(cols_to_exclude, axis=1, inplace=True)
test_X.drop(cols_to_exclude, axis=1, inplace=True)
#dev_X = dev_df.loc[:, dev_df.columns != 'totals.transactionRevenue']
#val_X = val_df.loc[:, val_df.columns != 'totals.transactionRevenue']
#dev_X = dev_X.loc[:, dev_X.columns != 'totals.totalTransactionRevenue']
#val_X = val_X.loc[:, val_X.columns != 'totals.totalTransactionRevenue']
#dev_X = dev_X.loc[:, dev_X.columns != 'totals.transactions']
#val_X = val_X.loc[:, val_X.columns != 'totals.transactions']
#dev_X = dev_X.loc[:, dev_X.columns != 'fullVisitorId']
#val_X = val_X.loc[:, val_X.columns != 'fullVisitorId']

#test_X = test_df.loc[:, test_df.columns != 'totals.transactionRevenue']
#test_X = test_X.loc[:, test_X.columns != 'totals.totalTransactionRevenue']
#test_X = test_X.loc[:, test_X.columns != 'totals.transactions']
#test_X = test_X.loc[:, test_X.columns != 'fullVisitorId']

#dev_X = dev_df[cat_cols + num_cols] 
#val_X = val_df[cat_cols + num_cols] 

#test_X = test_df[cat_cols + num_cols] 

channelGrouping
device.browser
device.deviceCategory
device.operatingSystem
geoNetwork.city
geoNetwork.continent
geoNetwork.country
geoNetwork.metro
geoNetwork.networkDomain
geoNetwork.region
geoNetwork.subContinent
trafficSource.adContent
trafficSource.adwordsClickInfo.adNetworkType
trafficSource.adwordsClickInfo.gclId
trafficSource.adwordsClickInfo.page
trafficSource.adwordsClickInfo.slot
trafficSource.campaign
trafficSource.keyword
trafficSource.medium
trafficSource.referralPath
trafficSource.source
trafficSource.adwordsClickInfo.isVideoAd
trafficSource.isTrueDirect


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:43: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:44: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:45: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior

In [31]:
#NaNs return error in random forest.
dev_X.fillna(0, inplace=True)
val_X.fillna(0, inplace=True)
test_X.fillna(0, inplace=True)

list(dev_X)

['channelGrouping',
 'device.browser',
 'device.deviceCategory',
 'device.isMobile',
 'device.operatingSystem',
 'fullVisitorId',
 'geoNetwork.city',
 'geoNetwork.continent',
 'geoNetwork.country',
 'geoNetwork.metro',
 'geoNetwork.networkDomain',
 'geoNetwork.region',
 'geoNetwork.subContinent',
 'totals.bounces',
 'totals.hits',
 'totals.newVisits',
 'totals.pageviews',
 'totals.sessionQualityDim',
 'totals.timeOnSite',
 'trafficSource.adContent',
 'trafficSource.adwordsClickInfo.adNetworkType',
 'trafficSource.adwordsClickInfo.gclId',
 'trafficSource.adwordsClickInfo.isVideoAd',
 'trafficSource.adwordsClickInfo.page',
 'trafficSource.adwordsClickInfo.slot',
 'trafficSource.campaign',
 'trafficSource.isTrueDirect',
 'trafficSource.keyword',
 'trafficSource.medium',
 'trafficSource.referralPath',
 'trafficSource.source',
 'visitId',
 'visitNumber',
 'visitStartTime',
 'year',
 'month',
 'day',
 'weekday']

In [32]:
a = np.intersect1d(dev_X.columns, test_X.columns)
print (a)
['channelGrouping' 'fullVisitorId' 'visitId' 'visitNumber'
 'visitStartTime' 'device.browser' 'device.deviceCategory'
 'device.isMobile' 'device.operatingSystem' 'geoNetwork.city'
 'geoNetwork.continent' 'geoNetwork.country' 'geoNetwork.metro'
 'geoNetwork.networkDomain' 'geoNetwork.region' 'geoNetwork.subContinent'
 'totals.bounces' 'totals.hits' 'totals.newVisits' 'totals.pageviews'
 'trafficSource.adContent' 'trafficSource.adwordsClickInfo.adNetworkType'
 'trafficSource.adwordsClickInfo.gclId'
 'trafficSource.adwordsClickInfo.isVideoAd'
 'trafficSource.adwordsClickInfo.page'
 'trafficSource.adwordsClickInfo.slot' 'trafficSource.campaign'
 'trafficSource.isTrueDirect' 'trafficSource.keyword'
 'trafficSource.medium' 'trafficSource.referralPath'
 'trafficSource.source' 'year' 'month' 'day' 'weekday']

['channelGrouping' 'day' 'device.browser' 'device.deviceCategory'
 'device.isMobile' 'device.operatingSystem' 'fullVisitorId'
 'geoNetwork.city' 'geoNetwork.continent' 'geoNetwork.country'
 'geoNetwork.metro' 'geoNetwork.networkDomain' 'geoNetwork.region'
 'geoNetwork.subContinent' 'month' 'totals.bounces' 'totals.hits'
 'totals.newVisits' 'totals.pageviews' 'totals.sessionQualityDim'
 'totals.timeOnSite' 'trafficSource.adContent'
 'trafficSource.adwordsClickInfo.adNetworkType'
 'trafficSource.adwordsClickInfo.gclId'
 'trafficSource.adwordsClickInfo.isVideoAd'
 'trafficSource.adwordsClickInfo.page'
 'trafficSource.adwordsClickInfo.slot' 'trafficSource.campaign'
 'trafficSource.isTrueDirect' 'trafficSource.keyword'
 'trafficSource.medium' 'trafficSource.referralPath'
 'trafficSource.source' 'visitId' 'visitNumber' 'visitStartTime' 'weekday'
 'year']


['channelGroupingfullVisitorIdvisitIdvisitNumbervisitStartTimedevice.browserdevice.deviceCategorydevice.isMobiledevice.operatingSystemgeoNetwork.citygeoNetwork.continentgeoNetwork.countrygeoNetwork.metrogeoNetwork.networkDomaingeoNetwork.regiongeoNetwork.subContinenttotals.bouncestotals.hitstotals.newVisitstotals.pageviewstrafficSource.adContenttrafficSource.adwordsClickInfo.adNetworkTypetrafficSource.adwordsClickInfo.gclIdtrafficSource.adwordsClickInfo.isVideoAdtrafficSource.adwordsClickInfo.pagetrafficSource.adwordsClickInfo.slottrafficSource.campaigntrafficSource.isTrueDirecttrafficSource.keywordtrafficSource.mediumtrafficSource.referralPathtrafficSource.sourceyearmonthdayweekday']

In [33]:
# custom function to run light gbm model
def run_lgb(train_X, train_y, val_X, val_y, test_X):
    params = {
        "objective" : "regression",
        "metric" : "rmse", 
        "num_leaves" : 30,
        "min_child_samples" : 100,
        "learning_rate" : 0.1,
        "bagging_fraction" : 0.7,
        "feature_fraction" : 0.5,
        "bagging_frequency" : 5,
        "bagging_seed" : 2018,
        "verbosity" : -1
    }
    
    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    model = lgb.train(params, lgtrain, 1000, valid_sets=[lgval], early_stopping_rounds=100, verbose_eval=100)
    
    pred_val_y = model.predict(val_X, num_iteration=model.best_iteration)
    
    pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    return model, pred_val_y, pred_test_y

# Training the model #
#pred_test, model, pred_val = run_lgb(dev_X, dev_y, val_X, val_y, test_X)
model, pred_val_lgb, pred_test_lgb = run_lgb(dev_X, dev_y, val_X, val_y, test_X)

ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in fields fullVisitorId

In [34]:
#Calculate RMSE Val
from sklearn import metrics
pred_val_lgb[pred_val_lgb<0] = 0
val_pred_lgb = pd.DataFrame({"fullVisitorId":val_df["fullVisitorId"].values})
val_pred_lgb["transactionRevenue"] = val_df["totals.transactionRevenue"].values
val_pred_lgb["PredictedRevenue"] = np.expm1(pred_val_lgb)
#print(np.sqrt(metrics.mean_squared_error(np.pred_val_lgb(val_pred_df["transactionRevenue"].values), np.log1p(val_pred_df["PredictedRevenue"].values))))
val_pred_lgb = val_pred_lgb.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()
print(np.sqrt(metrics.mean_squared_error(np.log1p(val_pred_lgb["transactionRevenue"].values), np.log1p(val_pred_lgb["PredictedRevenue"].values))))

NameError: name 'pred_val_lgb' is not defined

In [ ]:
#test
pred_test_lgb[pred_test_lgb<0] = 0
test_pred_lgb = pd.DataFrame({"fullVisitorId":test_df["fullVisitorId"].values})
test_pred_lgb["transactionRevenue"] = test_df["totals.transactionRevenue"].values
test_pred_lgb["PredictedRevenue"] = np.expm1(pred_test_lgb)
#print(np.sqrt(metrics.mean_squared_error(np.log1p(val_pred_df["transactionRevenue"].values), np.log1p(val_pred_df["PredictedRevenue"].values))))
test_pred_lgb = test_pred_lgb.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()
print(np.sqrt(metrics.mean_squared_error(np.log1p(test_pred_lgb["transactionRevenue"].values), np.log1p(test_pred_lgb["PredictedRevenue"].values))))

In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 100 decision trees

def run_rf(train_X, train_y, val_X, val_y, test_X):
    rf = RandomForestRegressor(n_estimators = 100, random_state = 42)
    rf.fit(train_X, train_y);
    
    pred_val_y = rf.predict(val_X)
    pred_test_y = rf.predict(test_X)
    return rf, pred_val_y, pred_test_y
# Train the model on training data
rf, pred_val_rf, pred_test_rf = run_rf(dev_X, dev_y, val_X, val_y, test_X);

In [ ]:
#validation
from sklearn import metrics
pred_val_rf[pred_val_rf<0] = 0
val_pred_rf = pd.DataFrame({"fullVisitorId":val_df["fullVisitorId"].values})
val_pred_rf["transactionRevenue"] = val_df["totals.transactionRevenue"].values
val_pred_rf["PredictedRevenue"] = np.expm1(pred_val_rf)
#print(np.sqrt(metrics.mean_squared_error(np.log1p(val_pred_df["transactionRevenue"].values), np.log1p(val_pred_df["PredictedRevenue"].values))))
val_pred_rf = val_pred_rf.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()
print(np.sqrt(metrics.mean_squared_error(np.log1p(val_pred_rf["transactionRevenue"].values), np.log1p(val_pred_rf["PredictedRevenue"].values))))

In [ ]:
#test
pred_test_rf[pred_test_rf<0] = 0
test_pred_rf = pd.DataFrame({"fullVisitorId":test_df["fullVisitorId"].values})
test_pred_rf["transactionRevenue"] = test_df["totals.transactionRevenue"].values
test_pred_rf["PredictedRevenue"] = np.expm1(pred_test_rf)
#print(np.sqrt(metrics.mean_squared_error(np.log1p(val_pred_df["transactionRevenue"].values), np.log1p(val_pred_df["PredictedRevenue"].values))))
test_pred_rf = test_pred_rf.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()
print(np.sqrt(metrics.mean_squared_error(np.log1p(test_pred_rf["transactionRevenue"].values), np.log1p(test_pred_rf["PredictedRevenue"].values))))